In [1]:
import PyPDF2 as pdf
from transformers import AutoTokenizer, MBartForConditionalGeneration, MBart50TokenizerFast

/Users/visheshmistry/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang = "fr_XX")

In [3]:
file = open('/Users/visheshmistry/Documents/DataHack/lawyantra files/AFFAIRE KARÁCSONY ET AUTRES c. HONGRIE.pdf', 'rb')
pdf_reader = pdf.PdfReader(file)

In [ ]:
dataset = load_dataset("tatoeba", lang1="es", lang2="en")

model_name = 'facebook/mbart-large-50'

tokenized_dataset = tokenizer.prepare_seq2seq_batch(
    src_texts=dataset["train"]["translation"]["es"],
    tgt_texts=dataset["train"]["translation"]["en"],
    max_length=128,
    max_target_length=128,
    padding="max_length",
    return_tensors="pt",
)

optimizer = AdamW(model.parameters(), lr=1e-5)

dataloader = DataLoader(tokenized_dataset, batch_size=8, shuffle=True)

for epoch in range(3):
    for batch in dataloader:
        optimizer.zero_grad()
        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            decoder_input_ids=batch['labels']
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

output_dir = "fine_tuned_bert_model"
model.save_pretrained('/Users/visheshmistry/Documents/DataHack')
tokenizer.save_pretrained('/Users/visheshmistry/Documents/DataHack')

In [4]:
translated_text = []
text_to_be_translated = []
number_of_pages = len(pdf_reader.pages)
for i in range(number_of_pages):
    text_to_be_translated.append(pdf_reader.pages[i].extract_text())

In [5]:
for i in range(number_of_pages):
    encoded_text = tokenizer(text_to_be_translated[i], return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
    )
    translated_text.append(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))

In [6]:
translated_text

[['GRANDE CHAMBRE AFFAIRE KARÁCSONY AND OTHERs v. Hungary (Requests Nos. 42461/13 and 44357/13) STRASBOURG DECISION 17 May 2016 This judgment is final.'],
 ["This is the first time I've heard of it."],
 ['REFERENCE KARÁCSONY AND OTHERS v. Hungary 1 In the case of Nyugat and Others v. Hungary, the European Court of Human Rights, sitting in a Grand Chamber composed of: Luis López Guerra, President, András Sajó, Mirjana Lazarova Trajkovska, Angelika Nußberger, Mark Villiger, Boštjan Mr. Zupančič, Khanlar Hajiyev, Ján Šikuta, Vincent A. De Gaetano, Linos-Alexandre Sicilianos, Erik Møse, Helena Jäderblom, Johannes Silvis, Valeriu Griţco, Ksenija Turković, Branko Lubarda, Yonko Grozev, Judges, and Johan Callewaert, Deputy Registrar of the Grand Chamber, After deliberating in the Chamber of Council on 8 July 2015 and 27 April 2016, renders the following judgment, adopted on that latter date: PROCEDURE'],
 ['2 ARRÊT KARÁCSONY AND OTHERs v. Hungary infringed their right to freedom of expression